In [1]:
import yfinance as yf

from quant.data_viz.stock_list import get_stocks_paginated, Query

# Load Finance Data

## Check data format

In [7]:
df = yf.download("AAPL", start="2020-01-01", end="2021-01-01")
df.head()

[*********************100%***********************]  1 of 1 completed


Price,Close,High,Low,Open,Volume
Ticker,AAPL,AAPL,AAPL,AAPL,AAPL
Date,,,,,
2020-01-02,72.620842,72.681289,71.373218,71.627092,135480400
2020-01-03,71.914818,72.676447,71.689957,71.847118,146322800
2020-01-06,72.487839,72.526526,70.783241,71.034702,118387200
2020-01-07,72.146935,72.753816,71.926907,72.497522,108872000
2020-01-08,73.307510,73.609745,71.849533,71.849533,132079200


## Get ticker by substring name

In [17]:
import requests


def get_companies_data(company_name):
    yfinance = "https://query2.finance.yahoo.com/v1/finance/search"
    user_agent = 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/108.0.0.0 Safari/537.36'
    params = {"q": company_name, "quotes_count": 5, "country": "United States"}

    res = requests.get(url=yfinance, params=params, headers={'User-Agent': user_agent})
    data = res.json()

    companies = data['quotes']
    return companies

In [18]:
get_companies_data(company_name="tesla")

[{'exchange': 'NMS',
  'shortname': 'Tesla, Inc.',
  'quoteType': 'EQUITY',
  'symbol': 'TSLA',
  'index': 'quotes',
  'score': 47512.0,
  'typeDisp': 'Equity',
  'longname': 'Tesla, Inc.',
  'exchDisp': 'NASDAQ',
  'sector': 'Consumer Cyclical',
  'sectorDisp': 'Consumer Cyclical',
  'industry': 'Auto Manufacturers',
  'industryDisp': 'Auto Manufacturers',
  'dispSecIndFlag': True,
  'isYahooFinance': True},
 {'exchange': 'NEO',
  'shortname': 'TESLA, INC. CDR (CAD HEDGED)',
  'quoteType': 'EQUITY',
  'symbol': 'TSLA.NE',
  'index': 'quotes',
  'score': 20037.0,
  'typeDisp': 'Equity',
  'longname': 'Tesla, Inc.',
  'exchDisp': 'NEO',
  'sector': 'Consumer Cyclical',
  'sectorDisp': 'Consumer Cyclical',
  'industry': 'Auto Manufacturers',
  'industryDisp': 'Auto Manufacturers',
  'isYahooFinance': True},
 {'exchange': 'GER',
  'shortname': 'Tesla Inc.                    R',
  'quoteType': 'EQUITY',
  'symbol': 'TL0.DE',
  'index': 'quotes',
  'score': 20022.0,
  'typeDisp': 'Equity',


In [12]:
from quant.data_viz.stock_list import get_stocks_paginated, Query

query = Query(name=None, page=1, sort="price_top", sector="Industrials", sub_industry=None)

# get_stocks_paginated(query)

In [2]:
from stock_list import load_sp500

df_sp500 = load_sp500()

In [7]:
unique_sector = df_sp500["GICS Sector"].unique().tolist()

In [8]:
tickers = df_sp500["Ticker"].tolist()

In [9]:
df_sp_500 = df_sp500

In [32]:
import pandas as pd

from pandas_datareader.stooq import StooqDailyReader

def get_last_price_and_dif(ticker: str) -> tuple[float, float]:
    """
    Возвращает последнюю доступную цену закрытия тикера через Stooq.
    """
    reader = StooqDailyReader(symbols=ticker, start=None, end=None, retry_count=3, pause=0.1)
    df = reader.read()  # :contentReference[oaicite:2]{index=2}
    if df.empty:
        raise ValueError(f"No data returned for ticker {ticker}")
    last_close = df["Close"].iloc[-1]
    pre_last_close = df["Close"].iloc[-30]
    return float(last_close), float((last_close - pre_last_close)/ last_close)


mask = pd.Series(True, index=df_sp_500.index)

if query.sector:
    mask &= df_sp_500["GICS Sector"].str.contains(query.sector)

if query.sub_industry:
    mask &= (df_sp_500["GICS Sub-Industry"] == query.sub_industry)

match = df_sp_500[mask].copy()
match["Price"] = None
match["Dif"] = None

for idx, row in match.iterrows():
    ticker = row["Ticker"]

    last_close, diff = get_last_price_and_dif(ticker)

    match.at[idx, "Price"] = last_close
    match.at[idx, "Dif"] = diff


match

KeyboardInterrupt: 

In [20]:
tickers = ["APPL", "TSLA", "NVDA"]

data = yf.download(
        tickers="APPL",
        period="1d",
        interval="1d",
        group_by="ticker",
        threads=True,
    )

[*********************100%***********************]  1 of 1 completed

1 Failed download:
['APPL']: YFRateLimitError('Too Many Requests. Rate limited. Try after a while.')


In [18]:
data

Empty DataFrame
Columns: [(NVDA, Open), (NVDA, High), (NVDA, Low), (NVDA, Close), (NVDA, Adj Close), (NVDA, Volume), (TSLA, Open), (TSLA, High), (TSLA, Low), (TSLA, Close), (TSLA, Adj Close), (TSLA, Volume), (APPL, Open), (APPL, High), (APPL, Low), (APPL, Close), (APPL, Adj Close), (APPL, Volume)]
Index: []

(78.3178, -0.1865961505558123)